## Немного о том, что можно делать с помощью питона с языком

In [ ]:
!pip install rnnmorph
!pip install natasha
!pip install razdel
!pip install transformers

Можно получать автоматическую морфо- и синтаксическую разметку. Для этого существует довольно много инструментов. Например, для русского языка есть такие:

In [ ]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)

In [ ]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

text = '29 марта 2017 года правительство Великобритании инициировало процедуру выхода в соответствии со статьёй 50 Договора о Европейском союзе; первоначально планировалось, что Великобритания покинет Европейский союз через два года, 29 марта 2019 года в 23:00 по Гринвичу.'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language='ru')
print([w.normal_form for w in predictor.predict(text.split())])

In [ ]:
sent.syntax.print()

Также можно автоматически извлекать имена, названия и другие подобные вещи (их называют именованными сущностями -- это важная задача в NLP)

In [ ]:
from natasha import NewsNERTagger

ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

Можно автоматически собирать большие коллекции текстов и обрабатывать их в удобном табличном виде:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
import os
import razdel

files = os.listdir('/content/drive/MyDrive/group_corpus')
texts = []
for file in files:
  if not file.endswith('txt'):
    continue
  path = os.path.join('/content/drive/MyDrive/group_corpus', file)
  with open(path) as f:
    texts.extend([sent.text for sent in razdel.sentenize(f.read())])
df = pd.DataFrame(texts)
pd.set_option('display.max_colwidth', None)
df.head()

Ну и наконец, можно играться с нейронными сетями, например, генерировать тексты на основе учебника Я.Г. Тестельца...

In [ ]:
%cd /content/drive/MyDrive/gpt2test 
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --k=5 \
    --p=0.95 \
    --length=70